In [ ]:
from main import *

m = treeSolution("13236073298", "Zjq030927")

In [21]:
import numpy as np
from PIL import Image
import cv2, time, ddddocr

def passComplexCaptcha(img, description):

    from ultralytics import YOLO

    img = cv2.imread(img)
    color_dict = {0: "蓝", 1: "灰", 2: "绿", 3: "红", 4: "黄"}

    complexModel = YOLO("data/passComplex.pt", task="detect", verbose=False)
    colorModel = YOLO("data/color.pt", task="classify", verbose=False)
    orcModel = ddddocr.DdddOcr(show_ad=False, use_gpu=True)
    result = complexModel.predict(img)[0].boxes
    segmentPosiveimg, segmentSideimg = [], []

    for index in range(len(result.xyxy)):
        x1, y1, x2, y2 = np.intp(result.xyxy[index].cpu())
        segmentImg = img[y1-5: y2+5, x1-5: x2+5]
        color = colorModel.predict(segmentImg, verbose=False)[0].probs.top1
        segmentImage = Image.fromarray(segmentImg)
        text = orcModel.classification(segmentImage)
        if text == "0": text = "o"
        if text == '' and "体" not in description: text = 'i'
        if "0" in description: description = description.replace("0", "o")

        if result.cls[index] == 0:
            segmentPosiveimg.append({text: [color_dict[color], (x1, y1, x2, y2)]})
        else: segmentSideimg.append({text: [color_dict[color], (x1, y1, x2, y2)]})

    alpha = [x for x in description if x.isascii()]
    # 仅考虑朝向
    if len(alpha) == 1 and "色" not in description:
        target = alpha[0].lower()
        if "侧" not in description:
            for item in segmentPosiveimg:
                key = list(item.keys())[0].lower()
                if key == target:
                    return item
        else:
            for item in segmentSideimg:
                key = list(item.keys())[0].lower()
                if key == target:
                    return item
    # 立方体乱入
    elif len(alpha) == 0 and "色" not in description and ("体" in description or "柱" in description or "锥" in description or "球" in description):
        segmentPosiveimg.extend(segmentSideimg)
        for item in segmentPosiveimg:
            key = list(item.keys())[0].lower()
            if key == '':
                return item
    # 仅考虑颜色
    elif len(alpha) == 1 and "向" not in description:
        target = alpha[0].lower()
        target_color = description.split("色")[0][-1]
        segmentPosiveimg.extend(segmentSideimg)
        for item in segmentPosiveimg:
            key = list(item.keys())[0].lower()
            color = list(item.values())[0][0]
            if key == target and color == target_color:
                return item
    # 立方体乱入
    elif len(alpha) == 0 and "向" not in description and ("体" in description or "柱" in description or "锥" in description or "球" in description):
        segmentPosiveimg.extend(segmentSideimg)
        for item in segmentPosiveimg:
            key = list(item.keys())[0].lower()
            if key == '':
                return item
    # 考虑朝向和颜色 alpha: 2
    elif len(alpha) == 2 and "向" in description and "色" in description:
        target = alpha[0].lower()
        target1 = alpha[1].lower()
        target_color = description.split("色")[0][-1]
        for item in segmentPosiveimg:
            key = list(item.keys())[0].lower()
            color = list(item.values())[0][0]
            if key == target and color == target_color:
                for item in segmentPosiveimg:
                    key = list(item.keys())[0].lower()
                    if key == target1:
                        return item
        for item in segmentSideimg:
            key = list(item.keys())[0].lower()
            color = list(item.values())[0][0]
            if key == target and color == target_color:
                for item in segmentSideimg:
                    key = list(item.keys())[0].lower()
                    if key == target1:
                        return item
    # 考虑颜色不考虑朝向
    elif len(alpha) == 2 and "色" in description and "向" not in description:
        target = alpha[0].lower()
        target1 = alpha[1].lower()
        segmentPosiveimg.extend(segmentSideimg)
        for item in segmentPosiveimg:
            key = list(item.keys())[0].lower()
            if key == target:
                target_color = list(item.values())[0][0]
                for item in segmentPosiveimg:
                    key = list(item.keys())[0].lower()
                    color = list(item.values())[0][0]
                    if key == target1 and color == target_color:
                        return item
    # 考虑朝向不考虑颜色
    elif len(alpha) == 2 and "向" in description and "色" not in description:
        target = alpha[0].lower()
        target1 = alpha[1].lower()
        for item in segmentPosiveimg:
            key = list(item.keys())[0].lower()
            if key == target:
                for item in segmentPosiveimg:
                    key = list(item.keys())[0].lower()
                    if key == target1:
                        return item
        for item in segmentSideimg:
            key = list(item.keys())[0].lower()
            if key == target:
                for item in segmentSideimg:
                    key = list(item.keys())[0].lower()
                    if key == target1:
                        return item
                    

In [ ]:
s = "请点击红色大写K朝向一样的小写g"
passComplexCaptcha("22.png", s) 

In [7]:
import json

def to_json(question, answer, title=""):

    f = open(f"data/{title}.json", "+w", encoding="utf-8")
    json.dump(dict(zip(question, answer)), f,
              indent=4, ensure_ascii=False)
    f.close()

In [171]:
import re

def getQuestionAnswer(driver, link, save=False, title=''):
    
    driver.get(link)
    time.sleep(1)
    question = []
    answer = []
    try:
        if len(driver.find_elements(By.TAG_NAME, "font")[0].text.split("\n")) <= 1 :
            fu_text = driver.find_elements(By.TAG_NAME, "font")[1].text.split("\n")
        else: fu_text = driver.find_elements(By.TAG_NAME, "font")[0].text.split("\n")
        fu_text = [x for x in fu_text if x != '']
        flag = True
        for index in range(len(fu_text)):
            ti = fu_text[index]
            if "第一章" not in  ti and flag: 
                continue
            else: flag = False
            if fu_text[index][0].isdigit(): 
                tmp = index
                question.append(fu_text[index])
            if fu_text[index][:4] in ["我的答案"] :
                length = index - tmp - 1
                true_answer = re.sub("[:]", "：",fu_text[index])
                true_answer = true_answer.split("：")
                if len(true_answer) == 1: 
                    true_answer = fu_text[index][-1]
                else:
                    true_answer = true_answer[-1]
                answer_dic = {(chr(65+length-v)):v for v in range(length, 0, -1)}
                if len(true_answer) == 1:
                    if true_answer in ["X", "错", 'x']: answer.append(0)
                    elif true_answer in ["√", "对"]: answer.append(1)
                    else:
                        option = answer_dic[true_answer]
                        answer.append(fu_text[index - option])
                else:
                    options = []
                    for option in true_answer:
                        option = answer_dic[option]
                        options.append(fu_text[index - option])
                    answer.append(options)
    except Exception as e:
        print(e)
        # return question, answer

    return question, answer

In [66]:
from main import *

option = wb.EdgeOptions()
option.add_argument('--headless')
driver = wb.Edge(options=option)
# driver.get('http://www.iamooc.com/zhihuishudaan.htm')

In [175]:
question, answers = getQuestionAnswer(driver, "http://www.iamooc.com/2000/060422.htm")

to_json(question, answers, title="汽车行走的艺术")


In [2]:
import cv2

def show(img):

    cv2.imshow("w", img)
    cv2.waitKey(-1)
    cv2.destroyAllWindows()


In [48]:
# 分割模块
"""
import random
import numpy as np

for i in index:

    bbox = boxes[i]
    x1, y1, w, h = np.intp(bbox)
    x2, y2 = x1 + w, y1 + h
    cv2.imwrite(f"colors/{random.randint(1, 1e6)}.jpg", img[y1: y2, x1: x2])
"""

In [20]:
# 重命名模块
"""
import os
from glob import glob

paths = glob("yzm/*.*")[1: ]
for index in range(0, len(paths)-1, 2):
    random_int = random.randint(1, 1e6)
    png = paths[index]
    txt = paths[index+1]

    os.rename(png, f"yzm\\{random_int}.png")
    os.rename(txt, f"yzm\\{random_int}.txt")
"""

In [49]:
# 数据集入表
"""
import pandas as pd
from glob import glob

df = pd.DataFrame({"img_path": glob("colors/*.*")})
df.to_csv("colors/colors.csv", index=False)
"""